Summarization 
Two ways to summarize or otherwise combine documents.

Stuff, which simply concatenates documents into a prompt;
Map-reduce, for larger sets of documents. This splits documents into batches, summarizes those, and then summarizes the summaries

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama3-8b-8192",groq_api_key=groq_api_key)

In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://mohammedfahimullah.medium.com/automate-your-angular-unit-testing-with-earlyai-e78e77f3fcbc")
docs = loader.load()
print(len(docs))

NameError: name 'YoutubeLoader' is not defined

In [4]:
from langchain.schema import Document

def trim_page_content(doc: Document, max_words: int) -> Document:
    words = doc.page_content.split() 
    trimmed_content = ' '.join(words[:max_words]) 
    return Document(page_content=trimmed_content, metadata=doc.metadata)

MAX_WORDS = 2000

trimmed_docs = [trim_page_content(doc, MAX_WORDS) for doc in docs]

for i, doc in enumerate(trimmed_docs):
    print(f"Document {i + 1} (Trimmed): {doc.page_content[:200]}...\n") 


Document 1 (Trimmed): Generate your Angular Unit Testing with EarlyAI | by Mohammedfahimullah | Dec, 2024 | MediumOpen in appSign upSign inWriteSign upSign inGenerate your Angular Unit Testing with EarlyAIStreamline Unit T...



I tried "stuff" to summarize a big document using groq llama model but the issue is the context size is less
so we cant use this. We have context models whose size is 128k tokens eg gpt-4o we can use these models
If you use models which have less content window you get issue 
"Please reduce the length of the messages or completion."



In [5]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [("system", "Write a concise summary of the following:\\n\\n{context}")]
)
# As discussed I am using stuff way of summarization.This is not a best practice
chain = create_stuff_documents_chain(llm, prompt)

result = chain.invoke({"context": trimmed_docs})
print(result)

Here is a concise summary of the article:

The author, Mohammedfahimullah, shares his experience with EarlyAI, a tool that uses AI to generate unit test cases for Angular projects. He finds the tool to be incredibly time-saving and simplifies the tedious task of writing unit tests. The tool generates test files for each method in the Angular component, covering happy paths and edge cases comprehensively. The author uses Jest as his testing framework, but EarlyAI also supports Mocha, Vitest, Pytest, and others. He notes that the tool provides documentation for the function and suggests potential code improvements. Overall, the author recommends giving EarlyAI a try and experiencing how it simplifies unit testing.
